# Neighborhood Based Recommender Systems
Here we implement a user-based recommender system on the MovieLens 100k Dataset.

In [1]:
# Importing packages
import numpy as np
import math
from scipy import sparse


## Movie Lens 1M
Here we work with the Movie Lens 1M data set, because the 20M one is a bit too big.

In [2]:
# setting up some constants
K_FOLD = 5

# numbers of users and items

ITEMS = 3952
USERS = 6040

In [3]:
# from random import randint

# # This function reads data from file and writes it to 
# def read_data(file, target):
#     with open(file) as f:
#         line = f.readline()
#         while line:
#             try:
#                 user, item, rating, _ = line.split(sep='::')
#                 target[randint(0,4)][int(user)-1, int(item)-1] = int(rating)
#             except ValueError:
#                 print(line)
#             line = f.readline()

# # Here we read in the ratings in ML-1M and store it in data_1m.npz

# data_1m = [sparse.lil_matrix((USERS, ITEMS), dtype=np.int8) for _ in range(K_FOLD)]

# read_data(".\\ml-1m\\ratings.dat", data_1m)

# for i in range(K_FOLD):
#     sparse.save_npz('data_1m_' + str(i) + ".npz", data_1m[i].tocsr())

In [4]:
# here we load the presplit datasets
data_1m = []

for i in range(K_FOLD):
    data_1m.append(sparse.load_npz("data_1m_" + str(i) + ".npz").astype(np.float64))

In [5]:
# split the data into training and testing sets
def get_training_set(i):
    return sum([data_1m[k] for k in range(K_FOLD) if k != i])

train_data_0 = get_training_set(0)
test_data_0 = data_1m[0]

In [6]:
# Generate the average rating for each user

averages = [user.sum() / user.count_nonzero() for user in train_data_0]

In [7]:
# def rated_items(user):
#     return set(train_data_0[user].nonzero()[1])

rated_items = [set(sparse.find(train_data_0[user])[1]) for user in range(USERS)]
rated_users = [sparse.find(train_data_0.getcol(item))[0] for item in range(ITEMS)]

In [34]:
import heapq

MAX_NEIGHBORS = 50

pearson_matrix = sparse.lil_matrix((USERS, USERS))

pred_vals = [sparse.lil_matrix((USERS, ITEMS)) for _ in range(MAX_NEIGHBORS)]

# This implements the Pearson metric as found in equation (2.2)
def pearson(user1, user2):
    if user1 > user2:
        temp = user1
        user1 = user2
        user2 = temp
    
    if pearson_matrix[user1, user2] != 0.0:
        if pearson_matrix[user1, user2] == -5:
            return 0
        return pearson_matrix[user1, user2]

    if user1 == user2:
        pearson_matrix[user1, user2] = 1
        return 1

    intersection = list(rated_items[user1].intersection(rated_items[user2]))

    if len(intersection) <= 10:
        pearson_matrix[user1, user2] = -5
        pearson_matrix[user2, user1] = -5
        return 0

    user1_ratings = train_data_0.getrow(user1).toarray()[0][intersection]
    user2_ratings = train_data_0.getrow(user2).toarray()[0][intersection]
    ones = np.ones_like(user1_ratings)
    user1_ratings = user1_ratings - ones * averages[user1]
    user2_ratings = user2_ratings - ones * averages[user2]
    bottom1 = math.sqrt(np.sum(np.power(user1_ratings, 2)))
    bottom2 = math.sqrt(np.sum(np.power(user2_ratings, 2)))

    if bottom1 * bottom2 == 0:
        pearson_matrix[user1, user2] = -5
        return 0

    top = np.dot(user1_ratings, user2_ratings)
    pearson_val = top / (bottom1 * bottom2)

    pearson_val = max(min(1, r), 0)

    pearson_matrix[user1, user2] = pearson_val

    return pearson_val

for user1 in range(USERS):

    if user1 % 10 == 0:
        print(user1)

    # rated_items_1 = rated_items(user1)
    for item in sparse.find(test_data_0[user1])[1]:

        pearson_user = []

        for user2 in rated_users[item]:
            pcc = pearson(user1, user2)
            if pcc >= 0:
                heapq.heappush(pearson_user, (-1 * pcc , user2))
        
        top = 0
        bot = 0
        avg = averages[user1]

        for k in range(MAX_NEIGHBORS):

            if len(pearson_user) <= k:
                if bot == 0:
                    for l in range(k, MAX_NEIGHBORS):
                        pred_vals[l][user1, item] = avg
                else:
                    for l in range(k, MAX_NEIGHBORS):
                        pred_vals[l][user1, item] = avg + top/bot           
                break

            pear, user2 = heapq.heappop(pearson_user)

            top += pear * (averages[user2] - train_data_0[user2,item])
            bot += abs(pear)

            if bot == 0:
                r = avg
            else:
                r = avg + top/bot

            r = max(min(5, r), 0)

            pred_vals[k][user1,item] = r
        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [35]:
# Here we re
# ad in the ratings in u1.test
for k in range(MAX_NEIGHBORS):
    # print(pred_vals[k].getrow(0))
    sparse.save_npz('pred_val_0_' + str(k) + '.npz', pred_vals[k].tocsr())


In [9]:
MAX_NEIGHBORS = 50
pred_vals = [sparse.load_npz('pred_val_0_' + str(k) + '.npz') for k in range(MAX_NEIGHBORS)]


In [10]:

# RMSE as found in equation 7.5
# Calculated using the average ratings of each user as a baseline



for k in range(MAX_NEIGHBORS):
    rms_error = math.sqrt((pred_vals[k] - test_data_0).power(2).sum() / test_data_0.count_nonzero())
    # s_error = (pred_vals[k] - test_data_0).power(2)
# 
    # rms_error = s_error.sum()

    print(k + 1, rms_error)

1 1.2848730201255207
2 1.1146291238527477
3 1.0573889615242704
4 1.0312122339661438
5 1.0109066977483927
6 0.9980042156336623
7 0.9880534033138556
8 0.9804914136090365
9 0.9740441253387581
10 0.969479546761973
11 0.9656689435474757
12 0.9627753298239798
13 0.9603576815955632
14 0.9579001729514034
15 0.9563056429538637
16 0.9550788858425708
17 0.9537618035859594
18 0.952708719196734
19 0.9512796653949559
20 0.9501946792523196
21 0.9492057174806815
22 0.9481883735501382
23 0.9474526350949062
24 0.9468173403685026
25 0.9462004048605518
26 0.9456452529545533
27 0.9451179917957556
28 0.9446350993927533
29 0.9439450606559351
30 0.9435265189468627
31 0.9430754540273619
32 0.9427570927397795
33 0.9423753128531357
34 0.9418879218301471
35 0.9415119985055668
36 0.9413472026952766
37 0.9410352730656976
38 0.9406976623554331
39 0.940362423875185
40 0.9401766836242261
41 0.9400085534111267
42 0.9398499642005982
43 0.9397090719957438
44 0.9396089920545112
45 0.9393957043550369
46 0.9391759381352526


In [12]:
from itertools import combinations

pred_vals = [sparse.load_npz('pred_val_0_' + str(k) + '.npz') for k in range(MAX_NEIGHBORS)]

for k, pred_val in enumerate(pred_vals, 1):
    print(k, pred_val)

1   (0, 0)	5.0
  (0, 149)	5.0
  (0, 587)	4.1891891891891895
  (0, 719)	5.0
  (0, 782)	3.5937996820349762
  (0, 1196)	5.0
  (0, 1544)	2.6932587760173967
  (0, 2017)	5.0
  (0, 2027)	4.652623211446741
  (0, 2339)	3.3833924422159716
  (0, 2686)	4.652623211446741
  (0, 2691)	5.0
  (0, 2790)	4.1891891891891895
  (0, 2796)	5.0
  (0, 2917)	5.0
  (0, 3104)	4.701392719249862
  (1, 291)	3.685274059105835
  (1, 355)	1.6437744950256254
  (1, 1197)	4.685274059105835
  (1, 1384)	2.6620073140999594
  (1, 1551)	3.1487080813633863
  (1, 1596)	4.467574652382037
  (1, 1686)	3.6620073140999594
  (1, 1791)	3.1487080813633863
  (1, 1833)	2.9142424641305777
  :	:
  (6039, 2019)	4.9050896954526495
  (6039, 2067)	3.08572695035461
  (6039, 2069)	2.458979638526653
  (6039, 2110)	2.6577797958830653
  (6039, 2133)	2.477212457600987
  (6039, 2247)	3.9050896954526495
  (6039, 2365)	3.98853702787399
  (6039, 2401)	1.9050896954526495
  (6039, 2438)	3.4542875564152165
  (6039, 2644)	5.0
  (6039, 2750)	3.952988855116515


In [ ]:
# try variety of neighborhood sizes
# try hitrate

In [34]:
squared_error_avg = 0

for user in range(USERS):
    if user % 100 == 0:
        print(user)
    for item in test_data_0[user].indices:
        
        if test_data_0[user, item] != 0:
            squared_error_avg += (averages[user] - test_data_0[user, item]) ** 2

RMSE_avg = math.sqrt(squared_error_avg / test_data_0.count_nonzero())

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000


In [ ]:
# Iterate through a range of neighborhood sizes
for K in range(2,21):
    # Predict ratings for each user-item pair in the test data.
    testing_size = 0
    error_count = 0
    u1_pred = np.zeros((USERS,ITEMS))
    for user in range(USERS):
        for item in range(ITEMS):
            if u1_test_data[user][item] == 0:
                continue
            try:
                pred_val = r_hat(user, item, K)
                u1_pred[user][item] = pred_val
                testing_size += 1
            except ZeroDivisionError:
                continue
            
    count = 0
    squared_error = 0

    # calculate the RMSE for each neighborhood size
    for user in range(USERS):
        for item in range(ITEMS):
            if u1_pred[user][item] == 0:
                continue
            squared_error += (u1_pred[user][item] - u1_test_data[user][item]) ** 2
            count += 1
    RMSE = math.sqrt(squared_error / count)

    # calculate the average Kendall rank correlation coefficient for each neighborhood size
    count = 0
    total_ken = 0
    for user in range(USERS):
        try:
            total_ken += user_kendall_coef(user)
            count += 1
        except ZeroDivisionError:
            continue

    print(K, RMSE, total_ken / count)